In [1]:
import os
import torch
import argparse
import warnings
import numpy as np
import anndata
from sklearn.preprocessing import OneHotEncoder
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
#import scDREAMER_train
from utils import *
from anndata import AnnData

In [2]:
def read_data(data_path, batch, cell_type, name, hvg=2000):

    Ann = sc.read_h5ad(data_path)
   
    b = Ann.obs[batch] #.to_list()
    batch_info = np.array([[i] for i in b]) 
    enc = OneHotEncoder(handle_unknown = 'ignore')

    enc.fit(batch_info.reshape(-1, 1))
    batch_info_enc = enc.transform(batch_info.reshape(-1, 1)).toarray()
    Ann.obsm[batch + '_encoded'] = pd.DataFrame(batch_info_enc).to_numpy()

    if cell_type is not None:
        
        labels = Ann.obs[cell_type].to_list()
        
        c = Ann.obs[cell_type]
        cell_info = np.array([[i] for i in c])

        enc.fit(cell_info.reshape(-1, 1))
        labels_enc = enc.transform(cell_info.reshape(-1, 1)).toarray()

        Ann.obsm[cell_type + "_encoded"] = pd.DataFrame(labels_enc).to_numpy()
       
    return Ann

In [3]:
data_path = "../Pan/Pancreas.h5ad"
batch = 'tech'
cell_type = 'celltype'
name = 'Pancreas'

adata = read_data(data_path, batch, cell_type, name)

In [ ]:
from torch.utils.data import Dataset, DataLoader


class AnndataLoader(Dataset):
    def __init__(self, data_path, batch):
        
        Ann = sc.read_h5ad(data_path)
        b = Ann.obs[batch] #.to_list()
        batch_info = np.array([[i] for i in b]) 
        enc = OneHotEncoder(handle_unknown = 'ignore')

        enc.fit(batch_info.reshape(-1, 1))
        batch_info_enc = enc.transform(batch_info.reshape(-1, 1)).toarray()
        Ann.obsm[batch + '_encoded'] = pd.DataFrame(batch_info_enc).to_numpy()
    
        self.Ann = Ann
        
    def __len__(self):
        return len(self.Ann)
        
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        x = self.Ann.X[idx, :]
        batch_encoded = self.Ann.obsm['tech_encoded'][idx, :]
        
        sample = {'X' : x, 
                  'batch_encoded' : batch_encoded} 
        
        return sample
        
         

In [ ]:
adata.X.shape
128*128

In [7]:
from torch.utils.data import Dataset, DataLoader


class AnndataLoader1(Dataset):
    def __init__(self, adata, batch):
        
        self.batch = batch    
        self.Ann = adata
        
    def __len__(self):
        return len(self.Ann)
        
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        x = self.Ann.X[idx, :]
        batch_encoded = self.Ann.obsm[self.batch + '_encoded'][idx, :]
        
        sample = {'X' : x, 
                  'batch_encoded' : batch_encoded} 
        
        return sample

In [8]:
ann_dataset = AnndataLoader1(adata, batch)
dataloader = DataLoader(ann_dataset, batch_size = 128, shuffle = True, num_workers = 0)

for i_batch, sample_batched in enumerate(dataloader):
    
    print (i_batch, sample_batched['X'].size(), sample_batched['batch_encoded'].size())
    sample = ann_dataset[i_batch]

0 torch.Size([128, 19093]) torch.Size([128, 9])
1 torch.Size([128, 19093]) torch.Size([128, 9])
2 torch.Size([128, 19093]) torch.Size([128, 9])
3 torch.Size([128, 19093]) torch.Size([128, 9])
4 torch.Size([128, 19093]) torch.Size([128, 9])
5 torch.Size([128, 19093]) torch.Size([128, 9])
6 torch.Size([128, 19093]) torch.Size([128, 9])
7 torch.Size([128, 19093]) torch.Size([128, 9])
8 torch.Size([128, 19093]) torch.Size([128, 9])
9 torch.Size([128, 19093]) torch.Size([128, 9])
10 torch.Size([128, 19093]) torch.Size([128, 9])
11 torch.Size([128, 19093]) torch.Size([128, 9])
12 torch.Size([128, 19093]) torch.Size([128, 9])
13 torch.Size([128, 19093]) torch.Size([128, 9])
14 torch.Size([128, 19093]) torch.Size([128, 9])
15 torch.Size([128, 19093]) torch.Size([128, 9])
16 torch.Size([128, 19093]) torch.Size([128, 9])
17 torch.Size([128, 19093]) torch.Size([128, 9])
18 torch.Size([128, 19093]) torch.Size([128, 9])
19 torch.Size([128, 19093]) torch.Size([128, 9])
20 torch.Size([128, 19093]) to

In [9]:
sample

{'X': array([0.       , 3.5244024, 0.       , ..., 0.       , 0.       ,
        2.01955  ], dtype=float32),
 'batch_encoded': array([0., 1., 0., 0., 0., 0., 0., 0., 0.])}